In [1]:
# Dependencies
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Additional Imports for ML model
from path import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split


In [2]:
fire_df = pd.read_csv("FW_Veg_Rem_Combined.csv")
fire_df.head(10)

,Unnamed: 0,Unnamed: 0.1,fire_name,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,disc_clean_date,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,0,0,NaN,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,2/11/2007,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,1,1,NaN,3.0,B,Arson,35.038330,-87.610000,TN,12/11/2006,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,2,2,NaN,60.0,C,Arson,34.947800,-88.722500,MS,2/29/2004,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,3,3,WNA 1,1.0,B,Debris Burning,39.641400,-119.308300,NV,6/6/2005,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,4,4,NaN,2.0,B,Miscellaneous,30.700600,-90.591400,LA,9/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633
5,5,5,NaN,1.0,B,Debris Burning,32.063900,-82.417800,GA,3/22/1999,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.139643
6,6,6,NaN,5.2,B,Debris Burning,31.316978,-83.393649,GA,1/10/2009,...,0.000000,79.896679,73.431818,0.000000,0.000000,26.0,0.0,0.0,0.0,0.148904
7,7,7,NaN,1.0,B,Campfire,30.904720,-93.557500,TX,11/12/2005,...,2.148857,72.899478,75.061381,77.924623,70.732911,28.4,27.5,1.2,55.4,0.241894
8,8,8,NaN,1.0,B,Arson,35.900310,-92.061180,AR,8/8/2007,...,2.103090,68.319022,67.575419,65.077844,60.196858,6.6,3.3,0.0,46.4,0.224629
9,9,9,CRAZY,1.0,B,Miscellaneous,48.839400,-99.718500,ND,4/19/2004,...,0.000000,64.606509,55.943038,54.337838,0.000000,12.3,1.8,0.0,0.0,0.291683


In [3]:
fire_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'fire_name', 'fire_size',
       'fire_size_class', 'stat_cause_descr', 'latitude', 'longitude', 'state',
       'disc_clean_date', 'cont_clean_date', 'discovery_month',
       'disc_date_final', 'cont_date_final', 'putout_time', 'disc_date_pre',
       'disc_pre_year', 'disc_pre_month', 'wstation_usaf', 'dstation_m',
       'wstation_wban', 'wstation_byear', 'wstation_eyear', 'Vegetation',
       'fire_mag', 'weather_file', 'Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7',
       'Temp_cont', 'Wind_pre_30', 'Wind_pre_15', 'Wind_pre_7', 'Wind_cont',
       'Hum_pre_30', 'Hum_pre_15', 'Hum_pre_7', 'Hum_cont', 'Prec_pre_30',
       'Prec_pre_15', 'Prec_pre_7', 'Prec_cont', 'remoteness'],
      dtype='object')

In [4]:
fire_df.dtypes

Unnamed: 0            int64
Unnamed: 0.1          int64
fire_name            object
fire_size           float64
fire_size_class      object
stat_cause_descr     object
latitude            float64
longitude           float64
state                object
disc_clean_date      object
cont_clean_date      object
discovery_month      object
disc_date_final      object
cont_date_final      object
putout_time          object
disc_date_pre        object
disc_pre_year         int64
disc_pre_month       object
wstation_usaf        object
dstation_m          float64
wstation_wban         int64
wstation_byear        int64
wstation_eyear        int64
Vegetation            int64
fire_mag            float64
weather_file         object
Temp_pre_30         float64
Temp_pre_15         float64
Temp_pre_7          float64
Temp_cont           float64
Wind_pre_30         float64
Wind_pre_15         float64
Wind_pre_7          float64
Wind_cont           float64
Hum_pre_30          float64
Hum_pre_15          

In [5]:
for column in fire_df.columns:
    print(f"Column {column} has {fire_df[column].isnull().sum()} null values")
    

Column Unnamed: 0 has 0 null values
Column Unnamed: 0.1 has 0 null values
Column fire_name has 29454 null values
Column fire_size has 0 null values
Column fire_size_class has 0 null values
Column stat_cause_descr has 0 null values
Column latitude has 0 null values
Column longitude has 0 null values
Column state has 0 null values
Column disc_clean_date has 0 null values
Column cont_clean_date has 27890 null values
Column discovery_month has 0 null values
Column disc_date_final has 26659 null values
Column cont_date_final has 29735 null values
Column putout_time has 27890 null values
Column disc_date_pre has 0 null values
Column disc_pre_year has 0 null values
Column disc_pre_month has 0 null values
Column wstation_usaf has 0 null values
Column dstation_m has 0 null values
Column wstation_wban has 0 null values
Column wstation_byear has 0 null values
Column wstation_eyear has 0 null values
Column Vegetation has 0 null values
Column fire_mag has 0 null values
Column weather_file has 0 nul

In [6]:
# use pandas-profiling to examine data

# !pip install pandas-profiling[notebook]
# from pandas_profiling import ProfileReport
# profile = ProfileReport(fire_df, title="Pandas Profiling Report-Wildfires")

# print to notebook 
# profile.to_widgets()

# print to file
# profile.to_file("reportdata.html")


In [7]:
# start to clean data

# check for duplicates
print(f"Duplicate entries: {fire_df.duplicated().sum()}")

Duplicate entries: 0


In [8]:
# remove columns. 

fire_df.drop(columns=["Unnamed: 0", "Unnamed: 0.1","fire_name",
                      "weather_file", "disc_clean_date", "cont_clean_date", 
                      "putout_time","disc_date_final","cont_date_final",
                      "disc_pre_month", "wstation_usaf", "disc_date_pre",
                      "dstation_m", "wstation_wban", "wstation_byear",
                    "wstation_eyear", "disc_pre_month"], inplace=True)
fire_df.head(10)


,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,discovery_month,disc_pre_year,Vegetation,fire_mag,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,Feb,2007,12,1.0,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,3.0,B,Arson,35.038330,-87.610000,TN,Dec,2006,15,0.1,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,60.0,C,Arson,34.947800,-88.722500,MS,Feb,2004,16,1.0,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,1.0,B,Debris Burning,39.641400,-119.308300,NV,Jun,2005,0,0.1,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,2.0,B,Miscellaneous,30.700600,-90.591400,LA,Sep,1999,12,0.1,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633
5,1.0,B,Debris Burning,32.063900,-82.417800,GA,Mar,1999,12,0.1,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.139643
6,5.2,B,Debris Burning,31.316978,-83.393649,GA,Jan,2008,12,0.1,...,0.000000,79.896679,73.431818,0.000000,0.000000,26.0,0.0,0.0,0.0,0.148904
7,1.0,B,Campfire,30.904720,-93.557500,TX,Nov,2005,12,0.1,...,2.148857,72.899478,75.061381,77.924623,70.732911,28.4,27.5,1.2,55.4,0.241894
8,1.0,B,Arson,35.900310,-92.061180,AR,Aug,2007,15,0.1,...,2.103090,68.319022,67.575419,65.077844,60.196858,6.6,3.3,0.0,46.4,0.224629
9,1.0,B,Miscellaneous,48.839400,-99.718500,ND,Apr,2004,15,0.1,...,0.000000,64.606509,55.943038,54.337838,0.000000,12.3,1.8,0.0,0.0,0.291683


In [9]:
fire_df.columns

Index(['fire_size', 'fire_size_class', 'stat_cause_descr', 'latitude',
       'longitude', 'state', 'discovery_month', 'disc_pre_year', 'Vegetation',
       'fire_mag', 'Temp_pre_30', 'Temp_pre_15', 'Temp_pre_7', 'Temp_cont',
       'Wind_pre_30', 'Wind_pre_15', 'Wind_pre_7', 'Wind_cont', 'Hum_pre_30',
       'Hum_pre_15', 'Hum_pre_7', 'Hum_cont', 'Prec_pre_30', 'Prec_pre_15',
       'Prec_pre_7', 'Prec_cont', 'remoteness'],
      dtype='object')

In [10]:
fire_df.head(5)

,fire_size,fire_size_class,stat_cause_descr,latitude,longitude,state,discovery_month,disc_pre_year,Vegetation,fire_mag,...,Wind_cont,Hum_pre_30,Hum_pre_15,Hum_pre_7,Hum_cont,Prec_pre_30,Prec_pre_15,Prec_pre_7,Prec_cont,remoteness
0,10.0,C,Missing/Undefined,18.105072,-66.753044,PR,Feb,2007,12,1.0,...,3.250413,78.216590,76.793750,76.381579,78.724370,0.0,0.0,0.0,0.0,0.017923
1,3.0,B,Arson,35.038330,-87.610000,TN,Dec,2006,15,0.1,...,2.122320,70.840000,65.858911,55.505882,81.682678,59.8,8.4,0.0,86.8,0.184355
2,60.0,C,Arson,34.947800,-88.722500,MS,Feb,2004,16,1.0,...,3.369050,75.531629,75.868613,76.812834,65.063800,168.8,42.2,18.1,124.5,0.194544
3,1.0,B,Debris Burning,39.641400,-119.308300,NV,Jun,2005,0,0.1,...,0.000000,44.778429,37.140811,35.353846,0.000000,10.4,7.2,0.0,0.0,0.487447
4,2.0,B,Miscellaneous,30.700600,-90.591400,LA,Sep,1999,12,0.1,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,0.214633


In [11]:
# Generate our categorical variable lists
fire_cat = fire_df.dtypes[fire_df.dtypes == "object"].index.tolist()
fire_cat

['fire_size_class', 'stat_cause_descr', 'state', 'discovery_month']

In [12]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(fire_df[fire_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(fire_cat)
encode_df.head()

,fire_size_class_B,fire_size_class_C,fire_size_class_D,fire_size_class_E,fire_size_class_F,fire_size_class_G,stat_cause_descr_Arson,stat_cause_descr_Campfire,stat_cause_descr_Children,stat_cause_descr_Debris Burning,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
# Merge one-hot encoded features and drop the originals

fire_df = fire_df.merge(encode_df, left_index=True, right_index=True)
fire_df = fire_df.drop(fire_cat,1)
fire_df.head(10)

,fire_size,latitude,longitude,disc_pre_year,Vegetation,fire_mag,Temp_pre_30,Temp_pre_15,Temp_pre_7,Temp_cont,...,discovery_month_Dec,discovery_month_Feb,discovery_month_Jan,discovery_month_Jul,discovery_month_Jun,discovery_month_Mar,discovery_month_May,discovery_month_Nov,discovery_month_Oct,discovery_month_Sep
0,10.0,18.105072,-66.753044,2007,12,1.0,24.480974,24.716923,24.902597,24.527961,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.0,35.038330,-87.610000,2006,15,0.1,7.553433,7.010000,0.343529,10.448298,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,60.0,34.947800,-88.722500,2004,16,1.0,4.971930,5.782766,5.558750,13.696600,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,39.641400,-119.308300,2005,0,0.1,16.275967,18.996181,18.142564,0.000000,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,2.0,30.700600,-90.591400,1999,12,0.1,-1.000000,-1.000000,-1.000000,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,1.0,32.063900,-82.417800,1999,12,0.1,-1.000000,-1.000000,-1.000000,-1.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,5.2,31.316978,-83.393649,2008,12,0.1,14.877341,16.409326,0.000000,0.000000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1.0,30.904720,-93.557500,2005,12,0.1,16.851939,16.997783,20.434783,11.985560,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,1.0,35.900310,-92.061180,2007,15,0.1,26.655241,27.264870,28.968064,28.682688,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1.0,48.839400,-99.718500,2004,15,0.1,4.600950,6.861878,6.053333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
fire_df.dtypes

fire_size              float64
latitude               float64
longitude              float64
disc_pre_year            int64
Vegetation               int64
                        ...   
discovery_month_Mar    float64
discovery_month_May    float64
discovery_month_Nov    float64
discovery_month_Oct    float64
discovery_month_Sep    float64
Length: 105, dtype: object

In [ ]:
fire_df.columns

In [15]:
len(fire_df)

55367

In [16]:
# Split our preprocessed data into our features and target arrays
y=fire_df.fire_size
X=fire_df.drop(columns="fire_size")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# # Create a random forest classifier.
# # rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# rf_model = RandomForestRegressor(random_state=78, n_estimators=100)
# rf_model = rf_model.fit(X_train_scaled, y_train)
# # print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
# # print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [22]:
# # Resample the training data with the BalancedRandomForestClassifier

# from imblearn.ensemble import BalancedRandomForestClassifier
# # Create a random forest classifier
# rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
# # Fit the model
# rf_model = rf_model.fit(X_train, y_train)
# rf_model
# # result is error -- ValueError: Unknown label type: 'continuous'

In [ ]:
# from imblearn.metrics import classification_report_imbalanced
# accuracy=classification_report_imbalanced(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [ ]:
# from sklearn.metrics import balanced_accuracy_score
# accuracy=balanced_accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [34]:
# # try neural
# import tensorflow as tf


# # Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# number_input_features = len(X_test_scaled[0])
# hidden_nodes_layer1 =  50
# hidden_nodes_layer2 = 10
# hidden_nodes_layer3 = 5


# nn = tf.keras.models.Sequential()

# # First hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# # Second hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# # Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# # Output layer
# nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# # Check the structure of the model
# nn.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 50)                5250      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_13 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 6         
Total params: 5,821
Trainable params: 5,821
Non-trainable params: 0
_________________________________________________________________


In [37]:
# # Compile the model
# nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
# # Fit the model
# fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
1298/1298 [==============================] - 1s 576us/step - loss: -489764768.0000 - accuracy: 0.1948
Epoch 2/50
1298/1298 [==============================] - 1s 586us/step - loss: -501627040.0000 - accuracy: 0.1948
Epoch 3/50
1298/1298 [==============================] - 1s 571us/step - loss: -513580288.0000 - accuracy: 0.1948
Epoch 4/50
1298/1298 [==============================] - 1s 571us/step - loss: -525757344.0000 - accuracy: 0.1948
Epoch 5/50
1298/1298 [==============================] - 1s 574us/step - loss: -537812160.0000 - accuracy: 0.1948
Epoch 6/50
1298/1298 [==============================] - 1s 572us/step - loss: -550074048.0000 - accuracy: 0.1948
Epoch 7/50
1298/1298 [==============================] - 1s 576us/step - loss: -562617856.0000 - accuracy: 0.1948
Epoch 8/50
1298/1298 [==============================] - 1s 662us/step - loss: -575296512.0000 - accuracy: 0.1948
Epoch 9/50
1298/1298 [==============================] - 1s 573us/step - loss: -588234688.0000 - 

In [38]:
# # Evaluate the model using the test data
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

433/433 - 0s - loss: -1.2969e+09 - accuracy: 0.2003
Loss: -1296883712.0, Accuracy: 0.20033232867717743
